In [44]:
!pip install scikit-learn -q

In [57]:
!pip install torchsummaryX wandb --quiet

In [71]:
import pandas as pd
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from torchsummaryX import summary
import wandb
from torch.utils.data import Dataset, DataLoader


In [70]:
import torch

if torch.backends.mps.is_available():
    print("MPS backend is available!")
else:
    print("MPS backend is not available.")
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

MPS backend is available!


In [25]:
df = pd.read_csv("merged_output2.csv")

In [26]:
df = df.dropna()

In [ ]:
na_rows = df[df.isna().any(axis=1)]

In [29]:
na_rows

,Unnamed: 0_x,Timestamp,People Count,RGB,Pico,Target


In [46]:
df

,Unnamed: 0_x,Timestamp,People Count,RGB,Pico,Target
0,6206.0,2025-05-02 11:28:08.000,4.0,4.0,5.0,5.0
1,6205.0,2025-05-02 11:28:07.000,3.0,4.0,5.0,4.0
2,6204.0,2025-05-02 11:28:06.000,3.0,4.0,5.0,4.0
3,6203.0,2025-05-02 11:28:05.000,2.0,3.0,5.0,4.0
4,6202.0,2025-05-02 11:28:03.000,2.0,3.0,5.0,4.0
...,...,...,...,...,...,...
6008,10.0,2025-04-28 16:04:14.000,4.0,4.0,4.0,4.0
6010,8.0,2025-04-28 16:04:12.000,4.0,5.0,4.0,4.0
6011,7.0,2025-04-28 16:04:11.000,4.0,4.0,4.0,4.0
6012,6.0,2025-04-28 16:04:10.000,5.0,4.0,4.0,4.0


In [39]:
len(training_data)

5569

In [47]:
x = df.iloc[:, 2:5]

In [48]:
y = df.iloc[:, -1]
y

0       5.0
1       4.0
2       4.0
3       4.0
4       4.0
       ... 
6008    4.0
6010    4.0
6011    4.0
6012    4.0
6013    4.0
Name: Target, Length: 5569, dtype: float64

# NN MLP

In [66]:
train, temp = train_test_split(df, test_size=0.3, random_state=42)
val, test = train_test_split(temp, test_size=0.5, random_state=42)
train = torch.tensor(train.values)
val = torch.tensor(val.values)
test = torch.tensor(test.values)


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [ ]:
class OccupancyDataset(Dataset):
    

In [40]:
config = {
    'activations': 'GELU',
    'learning_rate': 0.001,
    'max_lr' : 0.007,
    'pct_start': 0.45,
    'optimizers': 'AdamW',
    'scheduler': 'OneCycleLR', #'ReduceLROnPlateau'
    'epochs': 20,
    'batch_size': 32,
    'step_size' : 2,
    'weight_initialization': 'kaiming_normal', # e.g kaiming_normal, kaiming_uniform, uniform, xavier_normal or xavier_uniform
    'drop_pct':0.07
 }

In [ ]:
train_loader = torch.utils.data.DataLoader(
    dataset     = train,
    num_workers = 4,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = True,
)


val_loader = torch.utils.data.DataLoader(
    dataset     = validation_data,
    num_workers = 0,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

"val_loader = torch.utils.data.DataLoader(\n    dataset     = validation_data,\n    num_workers = 0,\n    batch_size  = config['batch_size'],\n    pin_memory  = True,\n    shuffle     = False\n)\n\n"

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers):
        super().__init__()
        self.sequential = nn.ModuleList()

        #input layer
        self.sequential.append(nn.Linear(input_dim, hidden_dim))
        self.sequential.append(nn.ReLU())

        #hidden
        for i in range(layers):
            self.sequential.append(nn.Linear(hidden_dim, hidden_dim))
            self.sequential.append(nn.ReLU())

        #output layer
        self.sequential.append(nn.Linear(hidden_dim, output_dim))

    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, torch.nn.Linear):
                if config["weight_initialization"] == "xavier_normal":
                    torch.nn.init.xavier_normal_(m.weight)
                elif config["weight_initialization"] == "xavier_uniform":
                    torch.nn.init.xavier_uniform_(m.weight)
                elif config["weight_initialization"] == "kaiming_normal":
                    torch.nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
                elif config["weight_initialization"] == "kaiming_uniform":
                    torch.nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
                elif config["weight_initialization"] == "uniform":
                    torch.nn.init.uniform_(m.weight)
                else:
                    raise ValueError("Invalid weight_initialization value")
                m.bias.data.fill_(0)
    def forward(self, x):
        for layer in self.sequential:
            x = layer(x)
        return y

In [65]:
model = MLP(3, 12, 1, 10)
summary(model, train)


TypeError: linear(): argument 'input' (position 1) must be Tensor, not DataFrame